In [62]:
import pandas as pd
import nltk
from nltk.collocations import *
from nltk.metrics.spearman import *

In [63]:
df = pd.read_csv('court-V-N.csv') # корпус размером в 1020 слов
rows = list(map(list, df.values))
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(rows)


In [75]:
#золотой стандарт в порядке по рангу (первая коллокация - самая сильная)
golden_stand = [('ВЫНЕСТИ', 'РЕШЕНИЕ'), ('ВЫНЕСТИ', 'ПРИГОВОР'), ('УДОВЛЕТВОРИТЬ', 'ХОДАТАЙСТВО'), 
                ('ВЫДАТЬ', 'САНКЦИЯ'), ('ПРИНЯТЬ', 'РЕШЕНИЕ'), 
                ('НАЛОЖИТЬ', 'АРЕСТ'), ('ПОДТВЕРДИТЬ', 'РЕШЕНИЕ'), ('ОТКЛОНИТЬ', 'ХОДАТАЙСТВО'), ('ОТКЛОНИТЬ', 'ИСК'), 
                ('ОБЖАЛОВАТЬ', 'ПРИГОВОР'), ('САНКЦИОНИРОВАТЬ', 'АРЕСТ')]


In [76]:
# топ 10 биграм с фильтром встречаемости не меньше 5 раз (метрика Log-Likelihood)
finder.apply_freq_filter(5)
top_lr = finder.nbest(bigram_measures.likelihood_ratio, 10)
print(top_lr)

[('СУД', 'ПРИЗНАТЬ'), ('ВЫДАТЬ', 'САНКЦИЯ'), ('СУД', 'УДОВЛЕТВОРИТЬ'), ('УДОВЛЕТВОРИТЬ', 'ИСК'), ('УДОВЛЕТВОРИТЬ', 'ХОДАТАЙСТВО'), ('ПРИНЯТЬ', 'РЕШЕНИЕ'), ('САНКЦИЯ', 'АРЕСТ'), ('МЕРА', 'ПРЕСЕЧЕНИЕ'), ('НАЛОЖИТЬ', 'АРЕСТ'), ('СУД', 'ВЫНЕСТИ')]


In [77]:
# топ 10 биграм с фильтром встречаемости не меньше 5 раз (метрика Pointwise Mutual Information)
finder.apply_freq_filter(5)
top_pmi = finder.nbest(bigram_measures.pmi, 10)
print(top_pmi)

[('ОСВОБОДИТЬ', 'ЗАЛОГ'), ('ЛИШЕНИЕ', 'СВОБОДА'), ('ПРОЙТИ', 'ПРЕНИЕ'), ('МЕРА', 'ПРЕСЕЧЕНИЕ'), ('ИЗБРАТЬ', 'МЕРА'), ('ВЫДАТЬ', 'САНКЦИЯ'), ('ОСТАВИТЬ', 'СИЛА'), ('АРЕСТОВАТЬ', 'АКЦИЯ'), ('СОГЛАСИТЬСЯ', 'ДОВОД'), ('НАЧАТЬСЯ', 'РАССМОТРЕНИЕ')]


In [78]:
#ранговый коэффициент корреляции Спирмена между ЗС и коллокациям по метрике Log-Likelihood
print('%0.1f' % spearman_correlation(ranks_from_sequence(golden_stand), ranks_from_sequence(top_lr)))

-0.8


In [79]:
#ранговый коэффициент корреляции Спирмена между ЗС и коллокациям по метрике Pointwise Mutual Information
print('%0.1f' % spearman_correlation(ranks_from_sequence(golden_stand), ranks_from_sequence(top_pmi)))

0.0


<b>(а)</b> какие коллокации, полученные методом i, не входят в Ваш золотой стандарт: 
в первом случае не входят коллокации: ('СУД', 'ПРИЗНАТЬ'), ('СУД', 'УДОВЛЕТВОРИТЬ'), ('УДОВЛЕТВОРИТЬ', 'ИСК'), ('САНКЦИЯ', 'АРЕСТ'), ('МЕРА', 'ПРЕСЕЧЕНИЕ'), ('СУД', 'ВЫНЕСТИ')
во втором случае: ('ОСВОБОДИТЬ', 'ЗАЛОГ'), ('ЛИШЕНИЕ', 'СВОБОДА'), ('ПРОЙТИ', 'ПРЕНИЕ'), ('МЕРА', 'ПРЕСЕЧЕНИЕ'), ('ИЗБРАТЬ', 'МЕРА'), ('ОСТАВИТЬ', 'СИЛА'), ('АРЕСТОВАТЬ', 'АКЦИЯ'), ('СОГЛАСИТЬСЯ', 'ДОВОД'), ('НАЧАТЬСЯ', 'РАССМОТРЕНИЕ') 

<b>(б)</b>  какие попали ошибочно (предложите объяснение - откуда взялась такая ошибка, что можно сделать, чтобы ее устранить): мне кажется, что ошибочно полала эта коллокации: ('ОСТАВИТЬ', 'СИЛА'). Возможно, что это просто частотная и даже сильная коллокация, но я смутно представляю, как это может быть связано с действиями суда. 

<b>(в)</b>  какие у Вас не попоали в ЗС, но вы думаете, что это полезные коллокации: 
('СУД', 'ПРИЗНАТЬ'),  ('СУД', 'УДОВЛЕТВОРИТЬ'), ('УДОВЛЕТВОРИТЬ', 'ИСК'), ('СУД', 'ВЫНЕСТИ'), ('ОСВОБОДИТЬ', 'ЗАЛОГ'), ('ЛИШЕНИЕ', 'СВОБОДА'), ('НАЧАТЬСЯ', 'РАССМОТРЕНИЕ')